## imports

In [ ]:
import os
import contextlib
import enum
import json
import os
import pathlib
import typing as tp
import uuid
from tqdm import tqdm
import copy
from collections import Counter
from  itertools import chain
import ast
import pandas as pd
import random


tqdm.pandas()

#nlp
from nltk import WordNetLemmatizer
from nltk.corpus import stopwords
from razdel import sentenize
import nltk
import re
import string

nltk.download('wordnet')
nltk.download('omw-1.4')
#nltk.download('stopwords')

## preprocessing

### courseworks se

In [ ]:
courseworks = pd.read_csv("data/se_spbu/works.csv")

In [ ]:
def string_list_to_list(string):
    return string[2:-1].split("', '")

def preprocessing_pipeline(text):
    text = string_list_to_list(text)
    text = " ".join(text[1:]) # введение
    text = re.sub("- ", "", text) # переносы строки
    
    formulas_characters = "@#^&*+_=<✓α>/≡≡Σ∈≤"
    chars = '●•'
    
    text = text.translate(str.maketrans('', '', chars))
    text = list(sentenize(text))
    
    
    sents = [sent.text for sent in text]
    sents = list(filter(lambda x: "аблица" not in x, sents))
    sents = list(filter(lambda x: "траница" not in x, sents))
    sents = list(filter(lambda x: "исунок" not in x, sents))
    sents = list(filter(lambda x: len(x)>10, sents))
    sents = list(filter(lambda x: len(x)<1500, sents))
    sents = list(filter(lambda x: not any(c in formulas_characters for c in x), sents))
    
    
    return sents

In [ ]:
courseworks["preprocessed_texts"] = courseworks.texts.apply(preprocessing_pipeline)
courseworks['preprocessed_len'] = courseworks.preprocessed_texts.apply(len)

In [ ]:
courseworks = courseworks[courseworks['preprocessed_len'] > 0]

In [ ]:
train_texts = courseworks[:-30]
val_texts = courseworks[-30:]
train_sentences = train_texts.preprocessed_texts.explode(ignore_index=True)
val_sentences = val_texts.preprocessed_texts.explode(ignore_index=True)

### habr

In [ ]:
habr = pd.read_csv("data/habr/clean_habr.csv",
                 lineterminator='\n')

In [ ]:
habr = habr[-5000:]

In [ ]:
def string_list_to_list(string):
    return string[2:-1].split("', '")

def preprocessing_pipeline_habr(text):
    formulas_characters = "@#^&*+_=<✓α>/≡≡Σ∈≤"
    chars = '●•'
    
    text = text.replace("\r", "\n ")
    text = text.replace("\n", " ")
    text = text.replace("\xa0", " ")
    text = text.translate(str.maketrans('', '', chars))
    text = list(sentenize(text))
    
    
    sents = [sent.text for sent in text]
    sents = list(filter(lambda x: "аблица" not in x, sents))
    sents = list(filter(lambda x: "траница" not in x, sents))
    sents = list(filter(lambda x: "исунок" not in x, sents))
    sents = list(filter(lambda x: len(x)>10, sents))
    sents = list(filter(lambda x: len(x)<1500, sents))
    sents = list(filter(lambda x: not any(c in formulas_characters for c in x), sents))
    
    
    return sents

In [ ]:
habr.text = habr.text.progress_apply(preprocessing_pipeline)

In [ ]:
train_texts = habr[:-300]
val_texts = habr[-300:]

#df_sentences = df.preprocessed_texts.explode(ignore_index=True)
train_sentences_habr = train_texts.text.explode(ignore_index=True)
val_sentences_habr = val_texts.text.explode(ignore_index=True)

In [ ]:
sentences_habr = make_text(sentences_habr)

## combine habr and courseworks

In [ ]:
train_habr = pd.DataFrame({"text": train_sentences_habr, "source" : "habr"})
val_habr = pd.DataFrame({"text": val_sentences_habr, "source" : "habr"})
train_courseworks = pd.DataFrame({"text": train_sentences, "source" : "coursework"})
val_courseworks = pd.DataFrame({"text": val_sentences, "source" : "coursework"})

In [ ]:
train = pd.concat([train_habr, train_courseworks])
val = pd.concat([val_habr, val_courseworks])

In [ ]:
train['target'] = train.source.apply(lambda x: 1 if x == "habr" else 0)
val['target'] = val.source.apply(lambda x: 1 if x == "habr" else 0)

In [ ]:
train.to_csv("habr_spbu290k_train.csv")
val.to_csv("habr_spbu290k_val.csv")

In [ ]:
train_df

## sentiment

## formality